Per prima cosa, caricare i file engine, coco_eval coco_utils, transforms e utils.
Caricare il file zip contentente il dataset PennFudan

In [ ]:
from zipfile import ZipFile

with ZipFile('/content/PennFudanPed.zip', 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image
import torchvision.transforms as T

class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))


    def __getitem__(self, idx):
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path)
        mask = np.array(mask)
        obj_ids = np.unique(mask)
        obj_ids = obj_ids[1:]
        masks = mask == obj_ids[:, None, None]
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
           img, target = self.transforms(img, target)
        
        return img, target

    def __len__(self):
        return len(self.imgs)

In [ ]:
import torch
from torch import nn


class Classifier(nn.Module):
    def __init__(self, input_channel, num_classes):
        super(Classifier, self).__init__()
        # Conf 1
        # classification layers (for scores)
        # self.classification = nn.Sequential(
        #     nn.Linear(input_channel, 10),
        #     nn.ReLU(),
        #     nn.Linear(10, 5),
        #     nn.ReLU(),
        #     nn.Linear(5, 2)
        # )
        # regression layers (for bounding boxes)
        # self.regression = nn.Sequential(
        #     nn.Linear(input_channel, 10),
        #     nn.ReLU(),
        #     nn.Linear(10, 5),
        #     nn.ReLU(),
        #     nn.Linear(5, num_classes * 4)
        # )

        # Conf 2
        # classification layers (for scores)
        self.classification = nn.Sequential(
            nn.Linear(input_channel, 20),
            nn.ReLU(),
            nn.Linear(20, 15),
            nn.ReLU(),
            nn.Linear(15, 10),
            nn.ReLU(),
            nn.Linear(10, 5),
            nn.ReLU(),
            nn.Linear(5, 2)
        )
        self.regression = nn.Sequential(
            nn.Linear(input_channel, 20),
            nn.ReLU(),
            nn.Linear(20, 15),
            nn.ReLU(),
            nn.Linear(15, 10),
            nn.ReLU(),
            nn.Linear(10, 5),
            nn.ReLU(),
            nn.Linear(5, num_classes * 4)
        )

    def forward(self, x):
        x = x.flatten(start_dim=1)
        scores = self.classification(x)
        bbox_coord = self.regression(x)
        # print('SCORES', scores, scores.shape)
        # print('BBOX_COORD', bbox_coord, bbox_coord.shape)
        return scores, bbox_coord

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
      
def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = Classifier(in_features, num_classes)
    return model

In [ ]:
from engine import train_one_epoch, evaluate
import utils
# import transforms as T
import torchvision.transforms as T

def get_transform(train):
    transforms = []  
    transforms.append(T.ToTensor())
    if train:
       transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [ ]:
dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))
dataset_test = PennFudanDataset('PennFudanPed', get_transform(train=False))

torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=2,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=2,
    collate_fn=utils.collate_fn)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# two classes - background and person
num_classes = 2

model = get_model(num_classes)
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# optimizer = torch.optim.ADAM(params, lr=0.005,
#                            momentum=0.9, weight_decay=0.0005)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()

    for images, targets in data_loader:
        # print(images, targets)    
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # during training the model's output is a dictionary with losses        
        loss_dict = model(images, targets)
        print(loss_dict)
        loss = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    lr_scheduler.step()
    evaluate(model, data_loader_test, device=device)

In [ ]:
img, _ = dataset_test[11]
to_pil = T.ToPILImage()
img = to_pil(img)
img.save("img.jpeg", "JPEG")

from PIL import Image, ImageFont, ImageDraw, ImageEnhance
import torchvision.transforms as T

img = Image.open('img.jpeg').convert('RGB')


preprocess = T.ToTensor()

x = preprocess(img)

draw = ImageDraw.Draw(img)

model.eval()
with torch.no_grad():
    prediction = model([x.to(device)])

for box in enumerate(prediction[0]['boxes']):
  print(box)
  draw.rectangle(((box[0], box[1]), (box[2], box[3])), outline="red", width=3)

Image.fromarray(preprocess(img).mul(255).permute(1, 2, 0).byte().numpy())
# source_img.save(out_file, "JPEG")

In [ ]:
torch.save(model.state_dict(), 'weights.pth')